<div style="text-align: right" align="right"><i>Peter Norvig, December 2024</i></div>

# Advent of Code 2024

I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, so here we go for 2024!  Our old friend [@GaryJGrady](https://x.com/garyjgrady) is here to provide illustrations:

<a href="https://x.com/garyjgrady"><img src="https://pbs.twimg.com/media/Gdp709FW8AAq2_m?format=jpg&name=medium" width=400 alt="GaryJGrady cartoon"></a>

I traditionally start by loading up my [**AdventUtils.ipynb**](AdventUtils.ipynb) notebook (same as last time except for the `current_year`):

In [1]:
%run AdventUtils.ipynb
current_year = 2024

Each day's solution consists of three parts, making use of my `parse` and `answer` utilities:
- **Reading the day's input**. E.g. `pairs = parse(1, ints)`. 
- **Solving Part One**. Find the solution and record it with, e.g., `answer(1.1, 4, lambda: 2 + 2)`.
- **Solving Part Two**. Find the solution and record it with, e.g., `answer(1.2, 9, lambda: 3 * 3)`.

The function `parse` assumes that the input is a sequence of records (default one per line), each of which should be parsed in some way (default just left as a string, but the argument `ints` says to treat each record as a tuple of integers). The function `answer` records the correct answer (for regression testing), and records the run time (that's why a `lambda:` is used).

To fully understand each day's puzzle, and to follow along the drama involving Santa, the elves, the Chief Historian, and all the rest, you need to read the descriptions on the [**AoC**](https://adventofcode.com/) site, as linked in the header for each of my day's solutions, e.g. [**Day 1**](https://adventofcode.com/2023/day/1) below. Since you can't read Part 2 until you solve Part 1, I'll partially describe Part 2 in this notebook. But I can't copy the content of AoC here, nor show my input files; you need to go to the site for that.



 

# [Day 1](https://adventofcode.com/2024/day/1) Historian Hysteria

According to the narrative, North Pole Historians created two lists of **location IDs**. We can parse them as a sequence of pairs of integers, and then use the transpose function, `T` to get two lists of ID numbers:

In [2]:
left, right = location_ids = T(parse(1, ints))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1000 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
38665   13337
84587   21418
93374   50722
68298   57474
54771   18244
49242   83955
66490   44116
65908   51323
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1000 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(38665, 13337)
(84587, 21418)
(93374, 50722)
(68298, 57474)
(54771, 18244)
(49242, 83955)
(66490, 44116)
(65908, 51323)
...


<img src="https://pbs.twimg.com/media/GdvPVOpXcAEZ34_?format=jpg&name=medium" width=400>

### Part 1: What is the total distance between your lists?

The **distance** between two numbers is the absolute value of their difference, and the **total distance** between two lists is the sum of the distances between respective pairs, where "respective" means to sort each list and then take the distance between the first element of each list, plus the distance between the second element of each list, and so on. (I use the transpose utility function, `T`, to turn the input sequence of 1000 pairs into two lists, each of 1000 integers.)

In [3]:
def total_distance(left: Ints, right: Ints) -> int:
    """Total distance between respective list elements, after sorting."""
    return sum(abs(a - b) for a, b in zip(sorted(left), sorted(right)))

answer(1.1, 1830467, lambda:
       total_distance(left, right))

Puzzle  1.1:   .0002 seconds, answer 1830467         ok

### Part 2: What is their similarity score?

The **similarity score** is the sum of each element of the left list times the number of times that value appears in the right list.

In [4]:
def similarity_score(left: Ints, right: Ints) -> int:
    """The sum of each x in `left` times the number of times x appears in `right`."""
    counts = Counter(right)
    return sum(x * counts[x] for x in left)

answer(1.2, 26674158, lambda:
       similarity_score(left, right))

Puzzle  1.2:   .0002 seconds, answer 26674158        ok

# [Day 2](https://adventofcode.com/2024/day/2): Red-Nosed Reports

Today's input is a sequence of **reports**, each of which is a sequence of integers:

In [5]:
reports = parse(2, ints)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1000 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
74 76 78 79 76
38 40 43 44 44
1 2 4 6 8 9 13
65 68 70 72 75 76 81
89 91 92 95 93 94
15 17 16 18 19 17
46 47 45 48 51 52 52
77 78 79 82 79 83
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1000 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(74, 76, 78, 79, 76)
(38, 40, 43, 44, 44)
(1, 2, 4, 6, 8, 9, 13)
(65, 68, 70, 72, 75, 76, 81)
(89, 91, 92, 95, 93, 94)
(15, 17, 16, 18, 19, 17)
(46, 47, 45, 48, 51, 52, 52)
(77, 78, 79, 82, 79, 83)
...


### Part 1: How many reports are safe?

A report is **safe** if it is  monotonically strictly increasing or strictly decreasing, and if no difference between adjacent numbers is greater than 3 in absolute value.

In [6]:
def safe(report: Ints) -> bool:
    """A report is safe if all differences are either in {1, 2, 3} or in {-1, -2, -3}."""
    deltas = diffs(report)
    return deltas.issubset({1, 2, 3}) or deltas.issubset({-1, -2, -3})
    
def diffs(report: Ints) -> set:
    """The set of differences between adjacent numbers in the report."""
    return {report[i] - report[i - 1] for i in range(1, len(report))}

assert diffs((7, 6, 4, 2, 1)) == {-1, -2}
assert safe((7, 6, 4, 2, 1))  == True

In [7]:
answer(2.1, 257, lambda:
       quantify(reports, safe))

Puzzle  2.1:   .0013 seconds, answer 257             ok

### Part 2: How many reports are safe using the Problem Dampener?

The **problem dampener** says that a report is safe if you can drop one element and get a safe report.

In [8]:
def safe_with_dampener(report: Ints) -> bool:
    """Is there any way to drop one element of `report` to get a safe report?"""
    return any(map(safe, drop_one(report)))

def drop_one(report) -> Iterable:
    """All ways of dropping one element of the input report."""
    return (report[:i] + report[i + 1:] for i in range(len(report)))

assert set(drop_one('1234')) == {'234', '134', '124', '123'}

In [9]:
answer(2.2, 328, lambda:
       quantify(reports, safe_with_dampener))

Puzzle  2.2:   .0076 seconds, answer 328             ok

# [Day 3](https://adventofcode.com/2024/day/3): Mull It Over

Today's input is a computer program with some corrupted characters. The program has multiple lines, but lines don't matter, so I will concatenate them into one big string:

In [10]:
program = cat(parse(3))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 6 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
where(536,162)~'what()what()how(220,399){ mul(5,253);mul(757,101)$where()@why()who()&when()from( ...
}?~who()select()-mul(316,505)&%*how()mul(363,589)>?%-:)where()~{{mul(38,452)select()%>[{]%>%mul( ...
?>where(911,272)'mul(894,309)~+%@#}@#why()mul(330,296)what()mul(707,884)mul;&}<{>where()$why()]m ...
> (when()[where()/#!/usr/bin/perl,@;mul(794,217)select():'])select()mul(801,192)why()&]why()/:]* ...
,+who():mul(327,845)/ >@[>@}}mul(86,371)!~&&~how(79,334)mul(637,103)why()mul(358,845)-#~?why(243 ...
where()#{*,!?:$mul(204,279)what()!{ what()mul(117,94)!select()>:mul(665,432)#don't()!!<!? mul(50 ...


### Part 1: What do you get if you add up all of the results of the multiplications?

For Part 1, just look for instructions of the form "mul(*digits*,*digits*)", using a regular expression and `re.findall`. Perform each of these multiplications and add them up, and ignore all other characters/instructions:

In [11]:
def execute(program: str) -> int:
    """The sum of the results of the multiply instructions."""
    return sum(prod(ints(m)) for m in multiplications(program))

def multiplications(program: str) -> List[str]:
    """A list of all the multiplication instructions in the program."""
    return re.findall(r'mul\(\d+,\d+\)', program)

test = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
assert execute(test) == 161
assert multiplications(test) == ['mul(2,4)', 'mul(5,5)', 'mul(11,8)', 'mul(8,5)']

In [12]:
answer(3.1, 156388521, lambda: 
       execute(program))

Puzzle  3.1:   .0015 seconds, answer 156388521       ok

### Part 2: What do you get if you add up all of the results of just the enabled multiplications?

For Part 2, the instruction "don't()"  says to disable (ignore) following multiply instructions until a "do()" instruction enables them again. I will define the function `enabled`, which returns the part of the program that is enabled, by susbstituting a space for the "don't()...do()" sequence.

In [13]:
def enabled(program: str) -> str:
    """Just the part of the program that is enabled; remove "don't()...do()" text."""
    return re.sub(r"don't\(\).*?(do\(\)|$)", " ", program)

test2 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
assert enabled(test2) == 'xmul(2,4)&mul[3,7]!^ ?mul(8,5))'
assert execute(enabled(test2)) ==  2 * 4 + 8 * 5 == 48
assert multiplications(enabled(test2)) == ['mul(2,4)', 'mul(8,5)']

In [14]:
answer(3.2, 75920122, lambda:
       execute(enabled(program)))

Puzzle  3.2:   .0009 seconds, answer 75920122        ok

# [Day 4](https://adventofcode.com/2024/day/4): Ceres Search

Today's puzzle is a 2D word-search puzzle:

In [15]:
xmas_grid = Grid(parse(4))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 140 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
MASAMXMSSXXMAMXXMXMASXMASXMMSMSMMMAXMASASMMSSMSXAXMASMMSMMMSSMSASMSSSSMSMSMXXMXMAXAMXMSMSSXSAMXM ...
MASMMXMASAXASMSMMMSAMXSMSAMXAAAAAXAMXASXAMAAAAMMSMMMMMASXAAAAMMAMAMMASAAAAXMXMSSSSSSMMSAMAXAXXSM ...
MMXAXMMMSXMAMAAXAAXAAAXXSMMSMSMSMXAXMXSMMMMSSMXAMXAAXMAMMMMSSMMAMAMMAMMMMMXSAAXAAMMAXXSAMXMSMAXM ...
SXSAMASASMSXMSMSMSSMMMMMMXAMXMMXMASMMMMAXXAAAMMMSSSSSMASXXAAXASMSXXMXSXSXSASMSMMSMSAMMMAMXAAMASX ...
AAAXXXMASASXMXMAXXMMASAASMXSASASXAAAAMSSMMMSXMAAMMMMMXAXMMMMSAMXAMASAMXSAMASXXAXAAMAMXSAMXSXSMMA ...
MSMMXXMMMAMAMMMMMMXSAXXAMMMMXSAXMMXXAMXAAMMXMASXMAAASMMXAAMXAXAMMMAMAMAMAMXMASXMMXMAAXMAXMAMXMSA ...
MXAXAMXXMMMMSAMAASMMMSMMASASAMAMAXMSXMSMMXAMXAXMMSSXSASXSSSMAMSMXMXSAMSSSMAMXMXAMAXXMMSAXAXMMXMA ...
ASXMMXSAMXAASXXMXSAAAXASAMMMASMSSSMAAMMXMMSSMASAMAMMMAMMAXMAXMASXM

### Part 1: How many times does XMAS appear?

We just have to find how many times the word "XMAS" appears in the grid, horizontally, vertically, or diagonally, forwards or backwards.  The variable `directions8` contains those eight directions (as (delta-x, delta-y) pairs). So examine each square of the grid and if it contains "X", see in how many of the directions it spells "XMAS". (Note that locations in the grid are denoted by `(x, y)` coordinates, as are directions (e.g., `(1, 0)` is the `East` direction. The functions `add` and `mul` do addition and scalar multiplication on these vectors.)

In [28]:
def word_search(grid: Grid, word='XMAS') -> int:
    """How many times does the given word appear in the grid?"""
    return quantify(grid_can_spell(grid, start, dir, word) 
                    for start in grid 
                    if grid[start] == word[0]
                    for dir in directions8)

def grid_can_spell(grid, start, dir, word):
    """Does `word` appear in grid starting at `start` and going in direction `dir`?"""
    return all(grid[add(start, mul(dir, i))] == word[i] for i in range(len(word)))

In [29]:
answer(4.1, 2401, lambda:
       word_search(xmas_grid))

Puzzle  4.1:   .1282 seconds, answer 2401            ok

### Part 1: How many times does an X-MAS appear?

Upon further review, the goal is not to find "XMAS" byt rather X-"MAS"; that is, two "MAS" words in an X pattern. The pattern can be any of these four:

     M.S     S.M     M.M     S.S
     .A.     .A.     .A.     .A.
     M.S     S.M     S.S     M.M

I decided to find these by looking for each instance of the middle letter ("A") in the grid, and then, for each pair of diagonal directions, see if the target word ("MAS") can be spelled in both directions:

In [18]:
diagonal_pairs = ([SE, NE], [SW, NW],  [SE, SW], [NE, NW])

def x_search(grid: Grid, word='MAS') -> int:
    """How many times does an X-MAS appear in the grid?"""
    return quantify((grid_can_spell(grid, sub(mid_pos, dir1), dir1, word) and
                     grid_can_spell(grid, sub(mid_pos, dir2), dir2, word))
                    for mid_pos in grid if grid[mid_pos] == word[1]
                    for dir1, dir2 in diagonal_pairs)

answer(4.2, 1822, lambda:
       x_search(xmas_grid))

Puzzle  4.2:   .0785 seconds, answer 1822            ok

# [Day 5](https://adventofcode.com/2024/day/5): Print Queue

Today's puzzle involves a **sleigh launch safety manual** that needs to be updated. The day's input is in two parts: the first a set of **rules** such as "47|53", which means that page 47 must be printed before page 53; and the second a list of **updates** of the form "75,47,61,53,29", meaning that those pages are to be printed in that order.

I mostly like my `parse` function, but I admit it is not ideal when an input file has two parts like this. I'll parse the two parts into paragraphs, and then call `parse` again on each paragraph:

In [19]:
manual  = parse(5, sections=paragraphs)
rules   = set(parse(manual[0], ints))
updates = parse(manual[1], ints)

assert (48, 39) in rules # `rules` is a set of (earlier, later) page number pairs
assert updates[0] == (61,58,51,32,12,14,71) # `updates` is a sequence of page number tuples

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1366 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
48|39
39|84
39|23
95|51
95|76
95|61
14|52
14|49
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 2 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
48|39
39|84
39|23
95|51
95|76
95|61
14|52
14|49
14|39
14|53
85|19
85|25
85|61
85|35
85|58
74|86
 ...
61,58,51,32,12,14,71
58,25,54,14,12,94,32,76,39
35,53,26,77,14,71,25,76,85,55,51,49,95
32,91,76, ...


### Part 1: What do you get if you add up the middle page number from the correctly-ordered updates?

I'll define `is_correct` to determine if an update is in the correct order, and `sum_of_correct_middles` to add up the middle numbers of the correct updates:

In [20]:
def sum_of_correct_middles(rules: Set[Ints], updates: Tuple[Ints]) -> int:
    """The sum of the middle elements of each update that is correct."""
    return sum(middle(update) for update in updates if is_correct(update, rules))

def is_correct(update: Ints, rules: Set[Ints]) -> bool:
    """An update is correct if no pair of pages violates a rule in the rules set."""
    return not any((second, first) in rules for (first, second) in combinations(update, 2))

def middle(sequence) -> object: return sequence[len(sequence) // 2]

In [21]:
answer(5.1, 5762, lambda:
       sum_of_correct_middles(rules, updates))

Puzzle  5.1:   .0010 seconds, answer 5762            ok

### Part 2: What do you get if you add up the middle page numbers after correctly re-ordering the incorrect updates?

In Part 2 we have to find the incorrect updates, re-order them into a correct order, and again sum the middle page numbers.
Since I have already defined `is_correct`, i could just generate all permutations of each update and find one that `is_correct`. That would work great if the longest update is only 5 pages long, as it is in the example input. But what is the longest update in the actual input?

In [22]:
max(map(len, updates))

23

That's not great. With 23 numbers there are 23! permutations, which is over 25 sextillion. So instead, here's my strategy:

- `sum_of_corrected_middles` will find the incorrect rules, perform a correction on each, and sum the middle numbers.
- `correction` will sort an update, obeying the rules. It used to be that Python's `sort` method allowed a `cmp` keyword to compare two values; there is vestigial support for this with the `functools.cmp_to_key` function. I will **sort** each update so that page *m* comes before page *n* if (*m*, *n*) is in the rules.
- Sorting will be a sextillion times faster than enumerating permutations.

In [23]:
def sum_of_corrected_middles(rules, updates) -> int:
    """The sum of the middle elements of each update that is correct."""
    incorrect = [update for update in updates if not is_correct(update, rules)]
    corrected = [correction(update, rules) for update in incorrect]
    return sum(map(middle, corrected))
    
def correction(update: Ints, rules) -> Ints:
    """Reorder the update to make it correct."""
    def rule_lookup(m, n): return +1 if (m, n) in rules else -1 
    return sorted(update, key=functools.cmp_to_key(rule_lookup))

In [24]:
answer(5.2, 4130, lambda:
       sum_of_corrected_middles(rules, updates))

Puzzle  5.2:   .0016 seconds, answer 4130            ok

I have to say, I'm pleased that this day I got both parts right the first time (and in fact, the same for the previous days). I was worried I might have my `+1` and `-1` backwards in `cmp_to_key`, but so far, everything has gone very smoothly. (However, even if I started solving right at midnight (which I don't), I don't think I would show up on the leaderboard; I have been good at getting a correct answer the first time, but I'm still way slower than the skilled contest programmers.

# Summary

So far, I've solved all the puzzles, each with a run time of less than a tenth of a second.

In [25]:
for d in sorted(answers):
    print(answers[d])

Puzzle  1.1:   .0002 seconds, answer 1830467         ok
Puzzle  1.2:   .0002 seconds, answer 26674158        ok
Puzzle  2.1:   .0013 seconds, answer 257             ok
Puzzle  2.2:   .0076 seconds, answer 328             ok
Puzzle  3.1:   .0015 seconds, answer 156388521       ok
Puzzle  3.2:   .0009 seconds, answer 75920122        ok
Puzzle  4.1:   .0990 seconds, answer 2401            ok
Puzzle  4.2:   .0785 seconds, answer 1822            ok
Puzzle  5.1:   .0010 seconds, answer 5762            ok
Puzzle  5.2:   .0016 seconds, answer 4130            ok
